<a href="https://colab.research.google.com/github/chirag-sharma-00/cs182-cv-project/blob/main/saved_model_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import glob
import pathlib
import tqdm
import os
import time
import copy
from __future__ import print_function
from __future__ import division
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from PIL import Image
from google.colab import drive
import shutil

In [2]:
!rm -rf sample_data

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Load Datasets

In [5]:
!unzip drive/MyDrive/182-cv-project/data/tiny-imagenet-200.zip
!tar -xvf drive/MyDrive/182-cv-project/data/tiny-imagenet-c.tar
!unzip drive/MyDrive/182-cv-project/data/imagenet-a.zip
!unzip drive/MyDrive/182-cv-project/data/imagenet-o.zip
!unzip drive/MyDrive/182-cv-project/data/speckle_augmented_val.zip

Streaming output truncated to the last 5000 lines.
  inflating: speckle_augmented_val/val_1478_1.JPEG  
  inflating: speckle_augmented_val/val_8603_2.JPEG  
  inflating: speckle_augmented_val/val_3674_1.JPEG  
  inflating: speckle_augmented_val/val_9138_0.JPEG  
  inflating: speckle_augmented_val/val_7615_2.JPEG  
  inflating: speckle_augmented_val/val_2570_0.JPEG  
  inflating: speckle_augmented_val/val_3126_0.JPEG  
  inflating: speckle_augmented_val/val_5597_1.JPEG  
  inflating: speckle_augmented_val/val_8573_2.JPEG  
  inflating: speckle_augmented_val/val_5949_1.JPEG  
  inflating: speckle_augmented_val/val_5823_2.JPEG  
  inflating: speckle_augmented_val/val_2399_0.JPEG  
  inflating: speckle_augmented_val/val_813_1.JPEG  
  inflating: speckle_augmented_val/val_5776_2.JPEG  
  inflating: speckle_augmented_val/val_7378_1.JPEG  
  inflating: speckle_augmented_val/val_6191_2.JPEG  
  inflating: speckle_augmented_val/val_1967_0.JPEG  
  inflating: speckle_augmented_val/val_4127_1.JPE

In [6]:
data_dir = pathlib.Path("tiny-imagenet-200")
imagenet_a_path = pathlib.Path('imagenet-a')
imagenet_o_path = pathlib.Path('imagenet-o')
tiny_imagenet_c_path = pathlib.Path('Tiny-ImageNet-C')
speckle_aug_path = pathlib.Path('speckle_augmented_val')

In [7]:
#image_count = len(list(data_dir.glob('**/*.JPEG')))
!rm tiny-imagenet-200/.DS_Store
!rm tiny-imagenet-200/train/.DS_Store
!rm tiny-imagenet-200/val/.DS_Store
!rm tiny-imagenet-200/test/.DS_Store

rm: cannot remove 'tiny-imagenet-200/.DS_Store': No such file or directory
rm: cannot remove 'tiny-imagenet-200/train/.DS_Store': No such file or directory
rm: cannot remove 'tiny-imagenet-200/val/.DS_Store': No such file or directory
rm: cannot remove 'tiny-imagenet-200/test/.DS_Store': No such file or directory


In [8]:
images = pd.read_csv("tiny-imagenet-200/words.txt", names = ['Id', 'labels'], sep = '\t')

In [9]:
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
CLASS_NAMES.sort()
num_classes = len(CLASS_NAMES)
print(len(CLASS_NAMES)) #should be 200
sum([cls in images['Id'].unique() for cls in CLASS_NAMES])

200


200

In [10]:
for item in imagenet_a_path.iterdir():
  if item.is_dir() and not item.name in CLASS_NAMES:
    shutil.rmtree(item)
for item in imagenet_o_path.iterdir():
  if item.is_dir() and not item.name in CLASS_NAMES:
    shutil.rmtree(item)
for name in CLASS_NAMES:
  if not pathlib.Path(os.path.join(imagenet_a_path, name)).is_dir():
    os.mkdir(os.path.join(imagenet_a_path, name))
  if not pathlib.Path(os.path.join(imagenet_o_path, name)).is_dir():
    os.mkdir(os.path.join(imagenet_o_path, name))

## Prediction Function

In [11]:
def predict(dataloaders, model): 
    """
    Run a forward pass (without caching data) for given model and return accuracy
    """
    if torch.cuda.is_available():
      model = model.cuda()

    accuracies = []
    model.eval()
    
    for phase in dataloaders.keys(): 
        counter = 0
        running_corrects = 0
        running_total = 0

        for inputs, labels in dataloaders[phase]: 
            inputs = inputs.to(device)
            labels.data = labels.data.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            counter += 1
            
            running_corrects += torch.sum(preds == labels.data)
            running_total += len(preds)
            
        phase_acc = running_corrects / running_total
        accuracies.append(phase_acc)

    return accuracies
            

## Dataloaders

In [12]:
# Custom dataloader based on https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
class ValidationDataset(Dataset):
  def __init__(self, annotations_file, img_dir, transform=None):
    self.img_labels = pd.read_csv(annotations_file, sep='\t', names=['image', 'label', 'x1', 'y1', 'x2', 'y2'])
    self.img_dir = img_dir
    self.transform = transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
    image = Image.open(img_path)
    image = image.convert('RGB')
    label = self.img_labels.iloc[idx, 1]
    label = np.where(label==CLASS_NAMES)[0][0]
    if self.transform:
        image = self.transform(image)
    return image, label

In [13]:
input_size = 299
batch_size = 32

In [14]:
data_transform = transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [15]:
image_datasets = {
    'imagenet-a': datasets.ImageFolder(imagenet_a_path, data_transform),
    'imagenet-o': datasets.ImageFolder(imagenet_o_path, data_transform),
    'speckle-aug-val': ValidationDataset("speckle_val_annotations.txt", speckle_aug_path, data_transform),
}
for dir in tiny_imagenet_c_path.iterdir():
  image_datasets["imagenet-c-"+dir.name+"-2"] = datasets.ImageFolder(os.path.join(dir, "2"), data_transform)
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                   batch_size=batch_size, 
                                                   shuffle=True, num_workers=2) 
                                                   for x in image_datasets.keys()}

## Test All Models

In [16]:
inception_model = torch.hub.load('pytorch/vision:v0.9.0', "inception_v3", pretrained=True)
inception_aux_in_ftrs = inception_model.AuxLogits.fc.in_features
inception_model.AuxLogits.fc = nn.Linear(inception_aux_in_ftrs, num_classes)
inception_in_ftrs = inception_model.fc.in_features
inception_model.fc = nn.Linear(inception_in_ftrs, num_classes)
resnet_model = torch.hub.load('pytorch/vision:v0.9.0', "resnet50", pretrained=True)
resnet_in_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(resnet_in_ftrs, num_classes)

Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /root/.cache/torch/hub/v0.9.0.zip
Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [17]:
models_folder = pathlib.Path("drive/MyDrive/final_models")

In [18]:
for item in models_folder.iterdir():
  if item.name.endswith(".pt"):
    if "inception" in item.name:
      model = inception_model
    elif "resnet50" in item.name:
      model = resnet_model
    else:
      print("ARCHITECTURE NOT FOUND")
      break
    print(item)
    model.load_state_dict(torch.load(item))
    accuracies = predict(dataloaders_dict, model)
    for i, key in enumerate(image_datasets.keys()):
      print(key + " accuracy = ", accuracies[i].item())

drive/MyDrive/final_models/oob-inception.pt
imagenet-a accuracy =  0.013865404762327671
imagenet-o accuracy =  0.24395160377025604
speckle-aug-val accuracy =  0.3610000014305115
imagenet-c-zoom_blur-2 accuracy =  0.2142999917268753
imagenet-c-pixelate-2 accuracy =  0.3967999815940857
imagenet-c-snow-2 accuracy =  0.3042999804019928
imagenet-c-impulse_noise-2 accuracy =  0.18439999222755432
imagenet-c-motion_blur-2 accuracy =  0.2759000062942505
imagenet-c-fog-2 accuracy =  0.37790000438690186
imagenet-c-jpeg_compression-2 accuracy =  0.33809998631477356
imagenet-c-shot_noise-2 accuracy =  0.21079999208450317
imagenet-c-contrast-2 accuracy =  0.27639999985694885
imagenet-c-gaussian_noise-2 accuracy =  0.16030000150203705
imagenet-c-defocus_blur-2 accuracy =  0.25509998202323914
imagenet-c-elastic_transform-2 accuracy =  0.33949998021125793
imagenet-c-glass_blur-2 accuracy =  0.17759999632835388
imagenet-c-frost-2 accuracy =  0.3669999837875366
imagenet-c-brightness-2 accuracy =  0.45909

In [19]:
torch.cuda.empty_cache()